In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
from effdet import create_model
from torchvision.models import convnext_tiny, efficientnet_b0, efficientnet_b3, efficientnet_b4

# 사용자 데이터셋 정의 (테스트용)
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(labels_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 테스트 데이터셋 및 DataLoader 설정
test_dataset = CustomImageDataset(img_dir='../data_centric/test', labels_file='./sample_submission.csv', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 로드 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ConvNeXt 모델 설정
convnext_model = convnext_tiny(pretrained=False)
convnext_model.classifier[2] = nn.Linear(convnext_model.classifier[2].in_features, 17)
convnext_model.load_state_dict(torch.load('best_model_convnext_epoch_10_val_loss_0.4171_f1_0.9971.pth'))
convnext_model = convnext_model.to(device)
convnext_model.eval()

# EfficientNet 모델 설정
efficientnet_model = efficientnet_b4(pretrained=False)

# classifier의 마지막 레이어 크기 조정
num_ftrs = efficientnet_model.classifier[1].in_features  # classifier의 두 번째 레이어가 Linear 레이어임
efficientnet_model.classifier[1] = nn.Linear(num_ftrs, 17)  # 17개 클래스에 맞게 분류기 조정

efficientnet_model.load_state_dict(torch.load('best_model_efficientnet_epoch_10_val_loss_0.4243_f1_0.9957.pth'))
efficientnet_model = efficientnet_model.to(device)
efficientnet_model.eval()

# 테스트 데이터에 대한 앙상블 예측 수행
all_predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        
        # 각 모델로 예측 수행
        convnext_outputs = convnext_model(images)
        efficientnet_outputs = efficientnet_model(images)
        # print(convnext_outputs)
        # print(efficientnet_outputs)
        # 예측 결과의 평균 앙상블 수행
        outputs = (convnext_outputs + efficientnet_outputs) / 2
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())

# 예측 결과를 DataFrame으로 저장
submission_df = pd.read_csv("./sample_submission.csv")
submission_df['target'] = all_predictions  # 예측 결과 할당

submission_filename = f"./submission_ensemble_convnext_efficientnet.csv"
submission_df.to_csv(submission_filename, index=False)
print(f"테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 {submission_filename}로 저장되었습니다.")


/data/ephemeral/home/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1329919/1125253963.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to co

테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 ./submission_ensemble_convnext_efficientnet.csv로 저장되었습니다.


In [6]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
from torchvision.models import convnext_tiny, efficientnet_b4

from efficientnet_pytorch import EfficientNet

# 사용자 데이터셋 정의 (테스트용)
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform_convnext=None, transform_efficientnet=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(labels_file)
        self.transform_convnext = transform_convnext
        self.transform_efficientnet = transform_efficientnet

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        
        image_convnext = self.transform_convnext(image) if self.transform_convnext else image
        image_efficientnet = self.transform_efficientnet(image) if self.transform_efficientnet else image
        
        return image_convnext, image_efficientnet

# ConvNeXt 데이터 전처리
transform_convnext = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientNet 데이터 전처리
transform_efficientnet = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 테스트 데이터셋 및 DataLoader 설정
test_dataset = CustomImageDataset(
    img_dir='../data_centric/test', 
    labels_file='./sample_submission.csv', 
    transform_convnext=transform_convnext, 
    transform_efficientnet=transform_efficientnet
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 로드 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ConvNeXt 모델 설정
convnext_model = convnext_tiny(pretrained=False)
convnext_model.classifier[2] = nn.Linear(convnext_model.classifier[2].in_features, 17)
convnext_model.load_state_dict(torch.load('best_model_convnext_epoch_10_val_loss_0.4171_f1_0.9971.pth'))
convnext_model = convnext_model.to(device)
convnext_model.eval()

# # EfficientNet 모델 설정
# efficientnet_model = efficientnet_b4(pretrained=False)
# num_ftrs = efficientnet_model.classifier[1].in_features
# efficientnet_model.classifier[1] = nn.Linear(num_ftrs, 17)
# efficientnet_model.load_state_dict(torch.load('best_model_amber-sweep-1.pth'))
# efficientnet_model = efficientnet_model.to(device)
# efficientnet_model.eval()


# EfficientNet 모델 설정
efficientnet_model = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_model._fc = nn.Linear(efficientnet_model._fc.in_features, 17)
state_dict_path1 = 'best_model_amber-sweep-1.pth'
efficientnet_model.load_state_dict(torch.load(state_dict_path1, map_location=device))
efficientnet_model = efficientnet_model.to(device)
efficientnet_model.eval()

# 테스트 데이터에 대한 앙상블 예측 수행
all_predictions = []
with torch.no_grad():
    for images_convnext, images_efficientnet in test_loader:
        images_convnext = images_convnext.to(device)
        images_efficientnet = images_efficientnet.to(device)
        
        # 각 모델로 예측 수행
        convnext_outputs = convnext_model(images_convnext)
        efficientnet_outputs = efficientnet_model(images_efficientnet)
        
        # 예측 결과의 평균 앙상블 수행
        outputs = (convnext_outputs + efficientnet_outputs) / 2
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())

# 예측 결과를 DataFrame으로 저장
submission_df = pd.read_csv("./sample_submission.csv")
submission_df['target'] = all_predictions  # 예측 결과 할당

submission_filename = f"./submission_ensemble_convnext_efficientnet_2.csv"
submission_df.to_csv(submission_filename, index=False)
print(f"테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 {submission_filename}로 저장되었습니다.")

/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1329919/1734576123.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `

Loaded pretrained weights for efficientnet-b4


/tmp/ipykernel_1329919/1734576123.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  efficientnet_model.load_state_dict(torch.load(state_dict_path1, map_location=device))


테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 ./submission_ensemble_convnext_efficientnet_2.csv로 저장되었습니다.


In [4]:
!pip install efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16429 sha256=ca5085777309eae37132172c83a02c2b9c0bd6d08e4693a4d969b679bfb941ac
  Stored in directory: /data/ephemeral/home/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [13]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import glob
from sklearn.metrics import f1_score
from effdet import create_model
from torchvision.models import convnext_tiny
from efficientnet_pytorch import EfficientNet

# Focal Loss 정의
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        log_prob = nn.functional.log_softmax(inputs, dim=1)
        prob = torch.exp(log_prob)
        
        target_log_prob = log_prob.gather(1, targets.view(-1, 1)).squeeze(1)
        target_prob = prob.gather(1, targets.view(-1, 1)).squeeze(1)
        
        focal_term = (1 - target_prob) ** self.gamma
        loss = -self.alpha * focal_term * target_log_prob
        
        return loss.mean()

# EfficientDet 기반 분류 모델 정의
class EfficientDetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientDetClassifier, self).__init__()
        self.model = create_model(
            'tf_efficientdet_d0',
            bench_task='train',
            num_classes=num_classes,
            pretrained=True
        )
        
        # 백본 고정
        self.backbone = self.model.model.backbone
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        # BiFPN 설정 및 출력 크기 추론
        self.bifpn = self.model.model.fpn
        self.pool = nn.AdaptiveAvgPool2d(1)

        # 임의의 입력으로 BiFPN 출력 피처 크기 추론
        dummy_input = torch.randn(1, 3, 512, 512)
        with torch.no_grad():
            features = self.bifpn(self.backbone(dummy_input))
        num_features = features[-1].shape[1]
        
        # 새로운 분류기 레이어
        self.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.bifpn(x)[-1]  # 마지막 BiFPN 레이어의 출력 사용
        x = self.pool(x).flatten(1)
        x = self.fc(x)
        return x

# 사용자 데이터셋 정의 (테스트용)
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform_convnext=None, transform_efficientnet=None, transform_efficientnet2=None, transform_efficientdet=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(labels_file)
        self.transform_convnext = transform_convnext
        self.transform_efficientnet = transform_efficientnet
        self.transform_efficientnet2 = transform_efficientnet2
        self.transform_efficientdet = transform_efficientdet

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        
        image_convnext = self.transform_convnext(image) if self.transform_convnext else image
        image_efficientnet = self.transform_efficientnet(image) if self.transform_efficientnet else image
        image_efficientnet2 = self.transform_efficientnet2(image) if self.transform_efficientnet2 else image
        image_efficientdet = self.transform_efficientdet(image) if self.transform_efficientdet else image
        
        return image_convnext, image_efficientnet, image_efficientnet2, image_efficientdet

# ConvNeXt 데이터 전처리
transform_convnext = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientNet 데이터 전처리
transform_efficientnet = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientNet2 데이터 전처리
transform_efficientnet2 = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientDet 데이터 전처리
transform_efficientdet = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 테스트 데이터셋 및 DataLoader 설정
test_dataset = CustomImageDataset(
    img_dir='../data_centric/test', 
    labels_file='./sample_submission.csv', 
    transform_convnext=transform_convnext, 
    transform_efficientnet=transform_efficientnet,
    transform_efficientnet2=transform_efficientnet2, 
    transform_efficientdet=transform_efficientdet
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 로드 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ConvNeXt 모델 설정
convnext_model = convnext_tiny(pretrained=False)
convnext_model.classifier[2] = nn.Linear(convnext_model.classifier[2].in_features, 17)
convnext_model.load_state_dict(torch.load('best_model_convnext_epoch_10_val_loss_0.4171_f1_0.9971.pth'))
convnext_model = convnext_model.to(device)
convnext_model.eval()

# EfficientNet 모델 설정
efficientnet_model = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_model._fc = nn.Linear(efficientnet_model._fc.in_features, 17)
efficientnet_model.load_state_dict(torch.load('best_model_amber-sweep-1.pth'))
efficientnet_model = efficientnet_model.to(device)
efficientnet_model.eval()

# EfficientNet2 모델 설정
efficientnet_model2 = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_model2._fc = nn.Linear(efficientnet_model2._fc.in_features, 17)
efficientnet_model2.load_state_dict(torch.load('best_model_new_2~3_model.pth'))
efficientnet_model2 = efficientnet_model2.to(device)
efficientnet_model2.eval()

# EfficientDet 모델 설정
efficientdet_model = EfficientDetClassifier(num_classes=17)
latest_model_path = './best_model_efficientdet_epoch_30_val_loss_0.0448_f1_0.9644.pth'
efficientdet_model.load_state_dict(torch.load(latest_model_path))
efficientdet_model = efficientdet_model.to(device)
efficientdet_model.eval()

# 테스트 데이터에 대한 앙상블 예측 수행
all_predictions = []
with torch.no_grad():
    for images_convnext, images_efficientnet, images_efficientnet2, images_efficientdet in test_loader:
        images_convnext = images_convnext.to(device)
        images_efficientnet = images_efficientnet.to(device)
        images_efficientnet2 = images_efficientnet2.to(device)
        images_efficientdet = images_efficientdet.to(device)
        
        # 각 모델로 예측 수행
        convnext_outputs = convnext_model(images_convnext)
        efficientnet_outputs = efficientnet_model(images_efficientnet)
        efficientnet_outputs2 = efficientnet_model2(images_efficientnet2)
        efficientdet_outputs = efficientdet_model(images_efficientdet)
        
        # 예측 결과의 평균 앙상블 수행
        outputs = (convnext_outputs + efficientnet_outputs + efficientnet_outputs2 + efficientdet_outputs) / 4
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())

# 예측 결과를 DataFrame으로 저장
submission_df = pd.read_csv("./sample_submission.csv")
submission_df['target'] = all_predictions  # 예측 결과 할당

submission_filename = "./submission_ensemble_convnext_efficientnet_efficientdet_efficientnet.csv"
submission_df.to_csv(submission_filename, index=False)
print(f"테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 {submission_filename}로 저장되었습니다.")


/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1329919/4069942408.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to 

Loaded pretrained weights for efficientnet-b4


/tmp/ipykernel_1329919/4069942408.py:145: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  efficientnet_model.load_state_dict(torch.load('best_model_amber-sweep-1.pth'))


Loaded pretrained weights for efficientnet-b4


/tmp/ipykernel_1329919/4069942408.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  efficientnet_model2.load_state_dict(torch.load('best_model_new_2~3_model.pth'))
/tmp/

테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 ./submission_ensemble_convnext_efficientnet_efficientdet_efficientnet.csv로 저장되었습니다.


In [14]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import glob
from sklearn.metrics import f1_score
from effdet import create_model
from torchvision.models import convnext_tiny, resnet50
from efficientnet_pytorch import EfficientNet

# Focal Loss 정의
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        log_prob = nn.functional.log_softmax(inputs, dim=1)
        prob = torch.exp(log_prob)
        
        target_log_prob = log_prob.gather(1, targets.view(-1, 1)).squeeze(1)
        target_prob = prob.gather(1, targets.view(-1, 1)).squeeze(1)
        
        focal_term = (1 - target_prob) ** self.gamma
        loss = -self.alpha * focal_term * target_log_prob
        
        return loss.mean()

# EfficientDet 기반 분류 모델 정의
class EfficientDetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientDetClassifier, self).__init__()
        self.model = create_model(
            'tf_efficientdet_d0',
            bench_task='train',
            num_classes=num_classes,
            pretrained=True
        )
        
        # 백본 고정
        self.backbone = self.model.model.backbone
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        # BiFPN 설정 및 출력 크기 추론
        self.bifpn = self.model.model.fpn
        self.pool = nn.AdaptiveAvgPool2d(1)

        # 임의의 입력으로 BiFPN 출력 피처 크기 추론
        dummy_input = torch.randn(1, 3, 512, 512)
        with torch.no_grad():
            features = self.bifpn(self.backbone(dummy_input))
        num_features = features[-1].shape[1]
        
        # 새로운 분류기 레이어
        self.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.bifpn(x)[-1]  # 마지막 BiFPN 레이어의 출력 사용
        x = self.pool(x).flatten(1)
        x = self.fc(x)
        return x

# 사용자 데이터셋 정의 (테스트용)
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform_convnext=None, transform_efficientnet=None, transform_efficientnet2=None, transform_efficientdet=None, transform_resnet=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(labels_file)
        self.transform_convnext = transform_convnext
        self.transform_efficientnet = transform_efficientnet
        self.transform_efficientnet2 = transform_efficientnet2
        self.transform_efficientdet = transform_efficientdet
        self.transform_resnet = transform_resnet

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        
        image_convnext = self.transform_convnext(image) if self.transform_convnext else image
        image_efficientnet = self.transform_efficientnet(image) if self.transform_efficientnet else image
        image_efficientnet2 = self.transform_efficientnet2(image) if self.transform_efficientnet2 else image
        image_efficientdet = self.transform_efficientdet(image) if self.transform_efficientdet else image
        image_resnet = self.transform_resnet(image) if self.transform_resnet else image
        
        return image_convnext, image_efficientnet, image_efficientnet2, image_efficientdet, image_resnet

# ConvNeXt 데이터 전처리
transform_convnext = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientNet 데이터 전처리
transform_efficientnet = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientNet2 데이터 전처리
transform_efficientnet2 = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# EfficientDet 데이터 전처리
transform_efficientdet = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# ResNet50 데이터 전처리
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 테스트 데이터셋 및 DataLoader 설정
test_dataset = CustomImageDataset(
    img_dir='../data_centric/test', 
    labels_file='./sample_submission.csv', 
    transform_convnext=transform_convnext, 
    transform_efficientnet=transform_efficientnet,
    transform_efficientnet2=transform_efficientnet2, 
    transform_efficientdet=transform_efficientdet,
    transform_resnet=transform_resnet
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 로드 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ConvNeXt 모델 설정
convnext_model = convnext_tiny(pretrained=False)
convnext_model.classifier[2] = nn.Linear(convnext_model.classifier[2].in_features, 17)
convnext_model.load_state_dict(torch.load('best_model_convnext_epoch_10_val_loss_0.4171_f1_0.9971.pth'))
convnext_model = convnext_model.to(device)
convnext_model.eval()

# EfficientNet 모델 설정
efficientnet_model = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_model._fc = nn.Linear(efficientnet_model._fc.in_features, 17)
efficientnet_model.load_state_dict(torch.load('best_model_amber-sweep-1.pth'))
efficientnet_model = efficientnet_model.to(device)
efficientnet_model.eval()

# EfficientNet2 모델 설정
efficientnet_model2 = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_model2._fc = nn.Linear(efficientnet_model2._fc.in_features, 17)
efficientnet_model2.load_state_dict(torch.load('best_model_new_2~3_model.pth'))
efficientnet_model2 = efficientnet_model2.to(device)
efficientnet_model2.eval()

# EfficientDet 모델 설정
efficientdet_model = EfficientDetClassifier(num_classes=17)
latest_model_path = './best_model_efficientdet_epoch_30_val_loss_0.0448_f1_0.9644.pth'
efficientdet_model.load_state_dict(torch.load(latest_model_path))
efficientdet_model = efficientdet_model.to(device)
efficientdet_model.eval()

# ResNet50 모델 설정
resnet_model = resnet50(pretrained=False)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 17)
resnet_model.load_state_dict(torch.load('best_resnet50_epoch_6_loss_0.4323_f1_0.9948_acc_0.9949_prec_0.9949_rec_0.9948.pth'))
resnet_model = resnet_model.to(device)
resnet_model.eval()

# 테스트 데이터에 대한 앙상블 예측 수행
all_predictions = []
with torch.no_grad():
    for images_convnext, images_efficientnet, images_efficientnet2, images_efficientdet, images_resnet in test_loader:
        images_convnext = images_convnext.to(device)
        images_efficientnet = images_efficientnet.to(device)
        images_efficientnet2 = images_efficientnet2.to(device)
        images_efficientdet = images_efficientdet.to(device)
        images_resnet = images_resnet.to(device)
        
        # 각 모델로 예측 수행
        convnext_outputs = convnext_model(images_convnext)
        efficientnet_outputs = efficientnet_model(images_efficientnet)
        efficientnet_outputs2 = efficientnet_model2(images_efficientnet2)
        efficientdet_outputs = efficientdet_model(images_efficientdet)
        resnet_outputs = resnet_model(images_resnet)
        
        # 예측 결과의 평균 앙상블 수행
        outputs = (convnext_outputs + efficientnet_outputs + efficientnet_outputs2 + efficientdet_outputs + resnet_outputs) / 5
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())

# 예측 결과를 DataFrame으로 저장
submission_df = pd.read_csv("./sample_submission.csv")
submission_df['target'] = all_predictions  # 예측 결과 할당

submission_filename = "./submission_ensemble_convnext_efficientnet_efficientdet_resnet.csv"
submission_df.to_csv(submission_filename, index=False)
print(f"테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 {submission_filename}로 저장되었습니다.")


/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/ephemeral/home/tf/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1329919/623635141.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `

Loaded pretrained weights for efficientnet-b4


/tmp/ipykernel_1329919/623635141.py:155: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  efficientnet_model.load_state_dict(torch.load('best_model_amber-sweep-1.pth'))


Loaded pretrained weights for efficientnet-b4


/tmp/ipykernel_1329919/623635141.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  efficientnet_model2.load_state_dict(torch.load('best_model_new_2~3_model.pth'))
/tmp/i

테스트 데이터에 대한 앙상블 예측이 완료되었습니다. 파일이 ./submission_ensemble_convnext_efficientnet_efficientdet_resnet.csv로 저장되었습니다.
